## *Aula 1: Limpeza de dados no Pyspark*

In [ ]:
!pip install pyspark

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
produtos = spark.read.csv('produtos.csv', header=True, inferSchema=True)
clientes = spark.read.csv('clientes.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('avaliacoes_pedido.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('pagamentos_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('pedidos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('vendedores.csv', header=True, inferSchema=True)

In [ ]:
# Lembre-se que é importante conhecer os dados então olhe todos os datasets, veja as colunas e observe a qualidade
produtos.show(3)

+--------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|       perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|            artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|    esporte_lazer|                  46

In [ ]:
# Quando existe muita informação dentro da linha é normal que o sistema moste apenas parte da informação automaticamente.
# Usando a fórmula 'truncate' é possivel mostrar todo o conteúdo
clientes.show(3, truncate=False)

+--------------------------------+--------------------------------+-----------+---------------------+--------------+
|id_cliente                      |id_unico_cliente                |cep_cliente|cidade_cliente       |estado_cliente|
+--------------------------------+--------------------------------+-----------+---------------------+--------------+
|06b8999e2fba1a1fbc88172c00ba8bc7|861eff4711a542e4b93843c6dd7febb0|14409      |franca               |SP            |
|18955e83d337fd6b2def6b18a428ac77|290c77bc529b7ac935b93aa66c333dc3|9790       |sao bernardo do campo|SP            |
|4e7b3e00288586ebd08712fdd0374a03|060e732b5b29e8181a18229c7b0b2b5e|1151       |sao paulo            |SP            |
+--------------------------------+--------------------------------+-----------+---------------------+--------------+
only showing top 3 rows


In [26]:
from pyspark.sql.functions import col, concat, lit, when, count, regexp_extract

In [ ]:
# Vamos aprender várias formas de fazer a mesma coisa:
clientes.select('id_cliente').show(1, truncate=False) # Pode-se apenas chamar pela coluna desejada
clientes.select(clientes['id_cliente']).show(1) # Pode-se usar o simbolo de colchetes com aspas
clientes.select(clientes.id_cliente).show(1) # Pode-se usar apenas o ponto

clientes.select(col('id_cliente')).show(1) # A melhor opção é usar a função "col" pois você pode além de selecionar também fazer operações

NameError: name 'clientes' is not defined

In [ ]:
# Lembrete: Sempre crie variáveis novas para que a base principal continue a mesma. Caso hajam erros a correção é mais simples

produtos_tratar_categoria_nulos = produtos.na.fill({'categoria_produto': 'Não especificado'}) # A função "na.fill" é usada para preencher campos sem valor

produtos_tratar_categoria_nulos.filter(col('categoria_produto') == 'Não especificado').count() # Código para fazer a contagem de valores que modificamos acima

610

In [ ]:
print("Total pedidos: ", pedidos.count()) # Sempre que modificar ou tratar a sua base de dados veja as mudanças feitas com a função count

pedidos_unicos = pedidos.dropDuplicates() # Função dropDuplicates para excluir linhas com valores iguais
print("Total pedidos unicos: ", pedidos_unicos.count())

pedidos_remocao_nulos = pedidos_unicos.na.drop() # Função na.drop para apagar linhas onde exista algum valor nulo
print("Total pedidos sem nulos: ", pedidos_remocao_nulos.count())

# É possível especificar a função na.drop para excluir apenas caso hajam valores nulos em colunas específicas
pedidos_remocao_nulos_em_colunas_especificas = pedidos_unicos.na.drop(subset=['id_pedido', 'id_cliente']) # Usando a subfunção subset
print("Total pedidos sem nulos nas colunas de id: ", pedidos_remocao_nulos_em_colunas_especificas.count())

Total pedidos:  99441
Total pedidos unicos:  99441
Total pedidos sem nulos:  96461
Total pedidos sem nulos nas colunas de id:  99441


In [ ]:
# Caso você queira usar a função na.fill em várias colunas ao mesmo tempo podemos fazer o seguinte:
colunas = ['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm'] # Crie uma lista com as colunas desejadas para substituição dos valores nulos

# Crie uma função para iterar cada coluna na lista colunas
for coluna in colunas:
    produtos = produtos.na.fill({coluna: 0})

In [ ]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f570a1c7740>>

## *Aula 2: Transforme Dados*

In [ ]:
from pyspark.sql.functions import upper

In [ ]:
vendedores = spark.read.csv('vendedores.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('itens_pedido.csv', header=True, inferSchema=True)

In [ ]:
vendedores.show(5)
vendedores.printSchema()

+--------------------+------------+-----------------+---------------+
|         id_vendedor|cep_vendedor|  cidade_vendedor|estado_vendedor|
+--------------------+------------+-----------------+---------------+
|3442f8959a84dea7e...|       13023|         campinas|             SP|
|d1b65fc7debc3361e...|       13844|       mogi guacu|             SP|
|ce3ad9de960102d06...|       20031|   rio de janeiro|             RJ|
|c0f3eea2e14555b6f...|        4195|        sao paulo|             SP|
|51a04a8a6bdcb23de...|       12914|braganca paulista|             SP|
+--------------------+------------+-----------------+---------------+
only showing top 5 rows
root
 |-- id_vendedor: string (nullable = true)
 |-- cep_vendedor: integer (nullable = true)
 |-- cidade_vendedor: string (nullable = true)
 |-- estado_vendedor: string (nullable = true)



In [ ]:
# A função withColumn pede o nome da coluna assim que for chamada, caso a coluna não exista ela a cria, se existir ela apenas substitui os valores
vendedores_tratados = vendedores.withColumn('cep_vendedor', vendedores.cep_vendedor.cast('string')) # 'cast' para mudar o tipo de dado na coluna toda

vendedores_tratados = vendedores_tratados.withColumn('cidade_vendedor_tratado', upper(vendedores['cidade_vendedor'])) # 'upper' para tornar maiúsculas

vendedores_tratados.show(5)
vendedores_tratados.printSchema()

+--------------------+------------+-----------------+---------------+-----------------------+
|         id_vendedor|cep_vendedor|  cidade_vendedor|estado_vendedor|cidade_vendedor_tratado|
+--------------------+------------+-----------------+---------------+-----------------------+
|3442f8959a84dea7e...|       13023|         campinas|             SP|               CAMPINAS|
|d1b65fc7debc3361e...|       13844|       mogi guacu|             SP|             MOGI GUACU|
|ce3ad9de960102d06...|       20031|   rio de janeiro|             RJ|         RIO DE JANEIRO|
|c0f3eea2e14555b6f...|        4195|        sao paulo|             SP|              SAO PAULO|
|51a04a8a6bdcb23de...|       12914|braganca paulista|             SP|      BRAGANCA PAULISTA|
+--------------------+------------+-----------------+---------------+-----------------------+
only showing top 5 rows
root
 |-- id_vendedor: string (nullable = true)
 |-- cep_vendedor: string (nullable = true)
 |-- cidade_vendedor: string (nullable

In [ ]:
itens_pedido.show(5)
itens_pedido.printSchema()

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio|preco|valor_frete|
+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|      13.29|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|      19.93|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|      17.87|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|      12.79|
|00042b26cf59d7ce6...|             1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|      18.14|
+--------------------+--------------+-------------------

In [ ]:
itens_pedido_tratados = itens_pedido.\
    withColumn('preco', col('preco').cast('float')).\
    withColumn('valor_frete', col('valor_frete').cast('float')).\
    withColumnRenamed('valor_frete', 'frete').\
    withColumn('valor_total', col('preco') + col('frete'))

itens_pedido_tratados.show(5)
itens_pedido_tratados.printSchema()

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio|preco|frete|valor_total|
+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|13.29|      72.19|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|19.93|     259.83|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|17.87|     216.87|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|12.79|  25.779999|
|00042b26cf59d7ce6...|             1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|18.14|     218.04|
+-------

In [ ]:
from pyspark.sql.functions import to_date, date_format

In [ ]:
itens_pedido_data = itens_pedido_tratados.withColumn('data', to_date(col('data_limite_envio')))

itens_pedido_data = itens_pedido_data.withColumn('data_br', date_format(col('data_limite_envio'), 'dd/MM/yyyy'))

itens_pedido_data = itens_pedido_data.withColumn('hora', date_format(col('data_limite_envio'), 'HH:mm:ss'))

itens_pedido_data.show(5)
itens_pedido_data.printSchema()

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----+-----------+----------+----------+--------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio|preco|frete|valor_total|      data|   data_br|    hora|
+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----+-----------+----------+----------+--------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|13.29|      72.19|2017-09-19|19/09/2017|09:45:35|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|19.93|     259.83|2017-05-03|03/05/2017|11:05:13|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|17.87|     216.87|2018-01-18|18/01/2018|14:48:30|
|00024acbcdf0a6daa...|             1|7634da152a4610f15...|9d7a1d34a505

In [4]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

In [ ]:
# Caso sua basa de dados esteja com datas em formato br que é incompativel por natureza com o tipo data, para transformar o campo em tio data será necessário:
# Converter a data br em um formato chamado timestamp
# Usar a função from_timestamp para transformar o valor timestamp em um formato compativel com o tipo data
# Por fim, na mesma linha em que usou a função from_timestamp use a função cast e especifíque que os valores dessa coluna são do tipo data

itens_pedido_data_reverso = itens_pedido_data.withColumn('timestamp', unix_timestamp(col('data_br'), 'dd/MM/yyyy'))

itens_pedido_data_reverso = itens_pedido_data_reverso.withColumn('data_formatada', from_unixtime('timestamp', 'yyyy-MM-dd')).withColumn('data_formatada', col('data_formatada').cast('date'))

itens_pedido_data_reverso.show(5)
itens_pedido_data_reverso.printSchema()

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----+-----------+----------+----------+--------+----------+--------------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio|preco|frete|valor_total|      data|   data_br|    hora| timestamp|data_formatada|
+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----+-----------+----------+----------+--------+----------+--------------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|13.29|      72.19|2017-09-19|19/09/2017|09:45:35|1505779200|    2017-09-19|
|00018f77f2f0320c5...|             1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|19.93|     259.83|2017-05-03|03/05/2017|11:05:13|1493769600|    2017-05-03|
|000229ec398224ef6...|             1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48

In [ ]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f570a1c7740>>

## *Aula 3: Integre Dados*

In [ ]:
produtos = spark.read.csv('produtos.csv', header=True, inferSchema=True)
clientes = spark.read.csv('clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('itens_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('pedidos.csv', header=True, inferSchema=True)

In [ ]:
join_pedidos_clientes = pedidos.join(clientes, pedidos['id_cliente'] == clientes['id_cliente']) # Lembre-se, se você não especificar o join será sempre o inner join
join_pedidos_clientes = pedidos.join(clientes, ['id_cliente']) # Quando o nome da coluna for igual, é possível fazer dessa forma

# JOINS: Inner, Outer, LeftOuter, RightOuter, FullOuter

pedidos.show(3)
clientes.show(3)
join_pedidos_clientes.show(3)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-08-08 08:55:23|      2018-08-08 13:50:00| 2018-08-17 18:06:29|  2018-09

In [ ]:
# Caso seja necessário você pode continuar adicionando joins após o primeiro join
join_itens_pedido = itens_pedido.join(pedidos, 'id_pedido').join(produtos, 'id_produto')
join_itens_pedido.show(3)

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+----

In [ ]:
concat_pedidos = pedidos.withColumn('id_pedido_cliente', concat(col('id_pedido'), lit('-'), col('id_cliente')))
concat_pedidos.show(3)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   id_pedido_cliente|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|e481f51cbdc54678b...|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|53cdb2fc8bc7dce0b...|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    del

In [ ]:
# Note que a coluna que criamos por meio de concatenação está em ultimo lugar na lista (normal), vamos mudar isso.

# Primeiro, crie uma lista com os nomes das colunas na ordem desejada
colunas = concat_pedidos.columns
print(colunas)

colunas.remove('id_pedido_cliente') # Removendo o nome da coluna
colunas.insert(0, 'id_pedido_cliente') # Readicionando o nome da coluna, só que na primeira posição agora

pedidos_reorganizar_df = concat_pedidos.select(colunas) # Select feito com a lista em ordem correta
pedidos_reorganizar_df.show(3)

['id_pedido', 'id_cliente', 'status_pedido', 'data_compra_pedido', 'data_aprovacao_pedido', 'data_envio_transportadora', 'data_entrega_cliente', 'data_estimada_entrega', 'id_pedido_cliente']
+--------------------+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|   id_pedido_cliente|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|53cdb2fc8bc7dce0b...|b0830f

In [ ]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7bef822c8740>>

## *Aula 4: Valide a qualidade dos dados*

In [5]:
# O formato do Schema da tabela itens pedido estava errado quando feito automaticamente, farei manualmente
from pyspark.sql.types import *

lista_campos = [
    StructField('id_pedido', StringType()),
    StructField('item_is_pedido', StringType()),
    StructField('id_produto', StringType()),
    StructField('id_vendedor', StringType()),
    StructField('data_limite_envio', StringType()),
    StructField('preco', FloatType()),
    StructField('valor_frete', FloatType())
]

schema_definido = StructType(lista_campos)

In [6]:
produtos = spark.read.csv('produtos.csv', header=True, inferSchema=True)
clientes = spark.read.csv('clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.schema(schema_definido).csv('itens_pedido.csv', header=True)
pedidos = spark.read.csv('pedidos.csv', header=True, inferSchema=True)

itens_pedido.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- item_is_pedido: string (nullable = true)
 |-- id_produto: string (nullable = true)
 |-- id_vendedor: string (nullable = true)
 |-- data_limite_envio: string (nullable = true)
 |-- preco: float (nullable = true)
 |-- valor_frete: float (nullable = true)



In [ ]:
itens_pedido = itens_pedido.withColumn('preco_valido', when(col('preco') > 0, 'Válido').otherwise('Inválido'))

select1_df = itens_pedido.select('id_pedido', 'id_produto', 'preco', 'preco_valido')
select1_df.show(3)

select1_df.filter(col('preco_valido') == 'Inválido').show()
# Aparentemente o .show() não é compatível na mesma linha em que voce declara uma nova variável

+--------------------+--------------------+-----+------------+
|           id_pedido|          id_produto|preco|preco_valido|
+--------------------+--------------------+-----+------------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...| 58.9|      Válido|
|00018f77f2f0320c5...|e5f2d52b802189ee6...|239.9|      Válido|
|000229ec398224ef6...|c777355d18b72b67a...|199.0|      Válido|
+--------------------+--------------------+-----+------------+
only showing top 3 rows
+---------+----------+-----+------------+
|id_pedido|id_produto|preco|preco_valido|
+---------+----------+-----+------------+
+---------+----------+-----+------------+



In [ ]:
status_permitido = ['created', 'approved', 'delivered', 'shipped', 'canceled', 'invoiced', 'processing']

pedidos = pedidos.withColumn('status_valido', when(col('status_pedido').isin(status_permitido), 'Válido').otherwise('Inválido'))

select2 = pedidos.select('id_pedido', 'status_pedido', 'status_valido')
select2.show(3)

pedidos.filter(col('status_valido') == 'Inválido').show(3) # É sempre bom aplicar o filter para ver quais não estão validados

+--------------------+-------------+-------------+
|           id_pedido|status_pedido|status_valido|
+--------------------+-------------+-------------+
|e481f51cbdc54678b...|    delivered|       Válido|
|53cdb2fc8bc7dce0b...|    delivered|       Válido|
|47770eb9100c2d0c4...|    delivered|       Válido|
+--------------------+-------------+-------------+
only showing top 3 rows
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|status_valido|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|8e24261a7e58791d1...|64a254d30eed42cd0...|  unavailable|2017-11-16 15

In [ ]:
for coluna in clientes.columns:
    clientes.select(count(col(coluna)).alias(coluna)).show() # Contagem de linhas totais

+----------+
|id_cliente|
+----------+
|     99441|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|           99441|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|      99441|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|         99441|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|         99441|
+--------------+



In [ ]:
for coluna in clientes.columns:
    clientes.select(count(when(col(coluna).isNull(), coluna)).alias(coluna)).show() # Contagem de nulos dentro de cada coluna

+----------+
|id_cliente|
+----------+
|         0|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|               0|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|          0|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|             0|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|             0|
+--------------+



In [9]:
n_nulls_pedidos = pedidos.select([count(when(col(c).isNull(), c)).alias(c) for c in pedidos.columns]) # Contagem de vezes que o nome da coluna aparece
n_nulls_pedidos.show()

+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+
|id_pedido|id_cliente|status_pedido|data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+
|        0|         0|            0|                 0|                  160|                     1783|                2965|                    0|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+



In [12]:
pedidos_integridade = pedidos.join(clientes, ['id_cliente'], 'left_anti') # Quais id's estão presentes na tabela pedidos.id_cliente que não existem na tabela cliente.id_cliente
pedidos_integridade.select('id_pedido', 'id_cliente').show()

pedidos_integridade = itens_pedido.join(produtos, ['id_produto'], 'left_anti') # Quais id's estão presentes na tabela itens_pedido.id_produto que não existem na tabela produto.id_produto
pedidos_integridade.select('id_pedido', 'id_produto').show()

+---------+----------+
|id_pedido|id_cliente|
+---------+----------+
+---------+----------+

+---------+----------+
|id_pedido|id_produto|
+---------+----------+
+---------+----------+



In [31]:
# Caso precise validar tipos string a função 'rlike' é uma ótima opção = https://regexr.com
clientes_validacao = clientes.withColumn('cep_valido', when(col('cep_cliente').rlike(r'^\d{5}$'), 'Válido').otherwise('Inválido'))
clientes_validacao.select('id_cliente', 'cep_cliente', 'cep_valido').show(5) # Para validar o CEP que precisa ter 5 caracteres

produtos_validaca0 = produtos.withColumn('id_produto_valido', when(col('id_produto').rlike(r'^[a-f0-9]{32}$'), 'Válido').otherwise('Inválido'))
produtos_validaca0.select('id_produto', 'id_produto_valido').show(5) # Para validar o id que precisa ser letas de A até F ou numeros de 0 até 9 totalizando 32 caracteres

+--------------------+-----------+----------+
|          id_cliente|cep_cliente|cep_valido|
+--------------------+-----------+----------+
|06b8999e2fba1a1fb...|      14409|    Válido|
|18955e83d337fd6b2...|       9790|  Inválido|
|4e7b3e00288586ebd...|       1151|  Inválido|
|b2b6027bc5c5109e5...|       8775|  Inválido|
|4f2d8ab171c80ec83...|      13056|    Válido|
+--------------------+-----------+----------+
only showing top 5 rows
+--------------------+-----------------+
|          id_produto|id_produto_valido|
+--------------------+-----------------+
|1e9e8ef04dbcff454...|           Válido|
|3aa071139cb16b67c...|           Válido|
|96bd76ec8810374ed...|           Válido|
|cef67bcfe19066a93...|           Válido|
|9dc1a7de274444849...|           Válido|
+--------------------+-----------------+
only showing top 5 rows
